In [16]:
import pandas as pd
import json
import geopandas as gpd
pd.set_option("max_colwidth", None)
pd.options.display.max_columns = None

In [45]:
from shapely import wkt


In [5]:
df_health1 = pd.read_csv("../data/raw/CDC_PLACES_LOCAL/PLACES__Local_Data_for_Better_Health__County_Data_2022_release_20240102.csv")

In [6]:
df_health1.head(2)

,Year,StateAbbr,StateDesc,LocationName,DataSource,Category,Measure,Data_Value_Unit,Data_Value_Type,Data_Value,Data_Value_Footnote_Symbol,Data_Value_Footnote,Low_Confidence_Limit,High_Confidence_Limit,TotalPopulation,LocationID,CategoryID,MeasureId,DataValueTypeID,Short_Question_Text,Geolocation
0,2020,US,United States,NaN,BRFSS,Prevention,Current lack of health insurance among adults ...,%,Crude prevalence,15.3,NaN,NaN,15.0,15.7,331449281,59,PREVENT,ACCESS2,CrdPrv,Health Insurance,NaN
1,2020,AL,Alabama,Talladega,BRFSS,Health Outcomes,Diagnosed diabetes among adults aged >=18 years,%,Crude prevalence,15.1,NaN,NaN,14.3,15.9,79985,1121,HLTHOUT,DIABETES,CrdPrv,Diabetes,POINT (-86.1759302 33.3693135)


In [64]:
df_health = df_health1[['StateAbbr','StateDesc','LocationName','Geolocation']]

In [18]:
df_health[df_health.LocationName.str.contains("Richmond", na=False)].drop_duplicates()

,StateAbbr,StateDesc,LocationName,Geolocation
246,NY,New York,Richmond,POINT (-74.1398995 40.5612628)
18686,GA,Georgia,Richmond,POINT (-82.0749982 33.3614863)
107970,NC,North Carolina,Richmond,POINT (-79.7556946 35.004636)
160335,VA,Virginia,Richmond,POINT (-76.7305609 37.9428936)
160557,VA,Virginia,Richmond City,POINT (-77.4760085 37.531399)


In [13]:
# Load the US counties GeoJSON file using GeoPandas
us_counties_geojson_url = 'https://www2.census.gov/geo/tiger/GENZ2021/shp/cb_2021_us_county_20m.zip'
us_counties = gpd.read_file(us_counties_geojson_url)

us_counties.head()

,STATEFP,COUNTYFP,COUNTYNS,AFFGEOID,GEOID,NAME,NAMELSAD,STUSPS,STATE_NAME,LSAD,ALAND,AWATER,geometry
0,13,239,00350547,0500000US13239,13239,Quitman,Quitman County,GA,Georgia,06,391703076,24158295,"POLYGON ((-85.14183 31.83926, -85.11403 31.893..."
1,18,111,00450376,0500000US18111,18111,Newton,Newton County,IN,Indiana,06,1040539827,4349506,"POLYGON ((-87.52665 41.16609, -87.39380 41.162..."
2,19,053,00465215,0500000US19053,19053,Decatur,Decatur County,IA,Iowa,06,1377569408,4138626,"POLYGON ((-94.01480 40.89703, -93.55654 40.898..."
3,24,013,01696228,0500000US24013,24013,Carroll,Carroll County,MD,Maryland,06,1159355859,13112464,"POLYGON ((-77.21702 39.72022, -76.99932 39.720..."
4,29,155,00758532,0500000US29155,29155,Pemiscot,Pemiscot County,MO,Missouri,06,1275841039,53915406,"POLYGON ((-89.96131 36.38880, -89.75215 36.386..."


In [30]:
us_counties[us_counties.NAME.str.contains("Charles City")]

,STATEFP,COUNTYFP,COUNTYNS,AFFGEOID,GEOID,NAME,NAMELSAD,STUSPS,STATE_NAME,LSAD,ALAND,AWATER,geometry
775,51,036,01480109,0500000US51036,51036,Charles City,Charles City County,VA,Virginia,06,473711548,55274680,"POLYGON ((-77.24966 37.38200, -77.22427 37.38022, -77.17732 37.49060, -77.09581 37.44029, -76.90412 37.40382, -76.90447 37.37767, -76.87444 37.36543, -76.88316 37.22306, -76.92157 37.22534, -76.97244 37.24850, -76.99600 37.30098, -77.06927 37.26974, -77.12469 37.30292, -77.27067 37.32076, -77.24966 37.38200))"


In [19]:
merged_df = pd.merge(df_health, us_counties, how='left', left_on=['LocationName', 'StateAbbr'], right_on=['NAME', 'STUSPS'])


In [21]:
merged_df.head(2)

,StateAbbr,StateDesc,LocationName,Geolocation,STATEFP,COUNTYFP,COUNTYNS,AFFGEOID,GEOID,NAME,NAMELSAD,STUSPS,STATE_NAME,LSAD,ALAND,AWATER,geometry
0,US,United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
1,AL,Alabama,Talladega,POINT (-86.1759302 33.3693135),01,121,00161586,0500000US01121,01121,Talladega,Talladega County,AL,Alabama,06,1.908347e+09,60874403.0,"POLYGON ((-86.50313 33.17914, -86.45803 33.24143, -86.35734 33.29692, -86.34111 33.35460, -86.37866 33.39098, -86.28200 33.50990, -86.23533 33.49453, -86.16948 33.61924, -86.19373 33.69785, -86.14556 33.67910, -86.05067 33.67459, -85.99493 33.58648, -85.79456 33.58557, -85.79605 33.55622, -85.80524 33.53519, -85.85189 33.49874, -85.90491 33.49865, -85.92376 33.39621, -85.97652 33.38187, -85.98029 33.29419, -86.11820 33.29632, -86.12057 33.19451, -86.17283 33.19568, -86.17437 33.10439, -86.49103 33.10294, -86.50313 33.17914))"


In [22]:
merged_df[merged_df.GEOID.isna()]

,StateAbbr,StateDesc,LocationName,Geolocation,STATEFP,COUNTYFP,COUNTYNS,AFFGEOID,GEOID,NAME,NAMELSAD,STUSPS,STATE_NAME,LSAD,ALAND,AWATER,geometry
0,US,United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
1403,US,United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
1602,US,United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
1659,US,United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
2283,US,United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
178196,VA,Virginia,Franklin City,POINT (-76.9413956 36.6840142),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
178198,VA,Virginia,Richmond City,POINT (-77.4760085 37.531399),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
178261,VA,Virginia,Roanoke City,POINT (-79.9581739 37.278458),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
178276,VA,Virginia,Richmond City,POINT (-77.4760085 37.531399),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None


In [24]:
merged_df[merged_df.LocationName.str.contains("Richmond",na=False)].tail()

,StateAbbr,StateDesc,LocationName,Geolocation,STATEFP,COUNTYFP,COUNTYNS,AFFGEOID,GEOID,NAME,NAMELSAD,STUSPS,STATE_NAME,LSAD,ALAND,AWATER,geometry
178077,VA,Virginia,Richmond,POINT (-76.7305609 37.9428936),51,760,01789073,0500000US51760,51760,Richmond,Richmond city,VA,Virginia,25,155205067.0,6849165.0,"POLYGON ((-77.59546 37.55575, -77.53239 37.59248, -77.53148 37.59179, -77.52309 37.58608, -77.51646 37.58167, -77.51620 37.58216, -77.51416 37.58157, -77.51325 37.58253, -77.49994 37.57540, -77.49353 37.58229, -77.49087 37.58012, -77.48237 37.57314, -77.47249 37.57869, -77.47385 37.58063, -77.47409 37.58181, -77.47722 37.58614, -77.47833 37.58756, -77.47921 37.58765, -77.47942 37.59081, -77.47913 37.59310, -77.47721 37.59790, -77.47815 37.59889, -77.47680 37.59854, -77.47090 37.59840, -77.46876 37.59862, -77.46606 37.59913, -77.46584 37.59676, -77.45448 37.59750, -77.45524 37.60178, -77.44758 37.60281, -77.44751 37.60222, -77.44361 37.60151, -77.43762 37.59869, -77.43762 37.59857, -77.38551 37.53517, -77.42074 37.44752, -77.59685 37.53465, -77.59546 37.55575))"
178078,VA,Virginia,Richmond,POINT (-76.7305609 37.9428936),51,159,01480164,0500000US51159,51159,Richmond,Richmond County,VA,Virginia,06,495925426.0,64458631.0,"POLYGON ((-76.93696 38.07707, -76.87706 38.12436, -76.76472 38.00198, -76.63423 37.96663, -76.55604 37.93152, -76.50783 37.83881, -76.56473 37.83729, -76.66818 37.77839, -76.78374 37.89548, -76.90903 37.98848, -76.93696 38.07707))"
178131,VA,Virginia,Richmond City,POINT (-77.4760085 37.531399),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
178198,VA,Virginia,Richmond City,POINT (-77.4760085 37.531399),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
178276,VA,Virginia,Richmond City,POINT (-77.4760085 37.531399),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None


In [26]:
def adjust_location_name(name):
    if isinstance(name, str) and " City" in name:
        return name.replace(" City", "")
    return name

# Adjust location names in df_health
df_health['AdjustedLocationName'] = df_health['LocationName'].apply(adjust_location_name)

# Perform the merge
merged_df2 = pd.merge(df_health, us_counties, how='left', left_on=['AdjustedLocationName', 'StateAbbr'], right_on=['NAME', 'STUSPS'])


/tmp/ipykernel_54666/3825965528.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_health['AdjustedLocationName'] = df_health['LocationName'].apply(adjust_location_name)


In [28]:
merged_df2[merged_df2.GEOID.isna()]

,StateAbbr,StateDesc,LocationName,Geolocation,AdjustedLocationName,STATEFP,COUNTYFP,COUNTYNS,AFFGEOID,GEOID,NAME,NAMELSAD,STUSPS,STATE_NAME,LSAD,ALAND,AWATER,geometry
0,US,United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
402,VA,Virginia,Charles City,POINT (-77.05417 37.3610537),Charles,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
1403,US,United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
1602,US,United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
1659,US,United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
178159,VA,Virginia,James City,POINT (-76.7778878 37.3248367),James,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
178396,VA,Virginia,Charles City,POINT (-77.05417 37.3610537),Charles,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
178694,VA,Virginia,Charles City,POINT (-77.05417 37.3610537),Charles,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
178718,VA,Virginia,James City,POINT (-76.7778878 37.3248367),James,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None


In [29]:
merged_df2[merged_df2.LocationName.str.contains("Richmond",na=False)].tail()

,StateAbbr,StateDesc,LocationName,Geolocation,AdjustedLocationName,STATEFP,COUNTYFP,COUNTYNS,AFFGEOID,GEOID,NAME,NAMELSAD,STUSPS,STATE_NAME,LSAD,ALAND,AWATER,geometry
178485,VA,Virginia,Richmond City,POINT (-77.4760085 37.531399),Richmond,51,159,01480164,0500000US51159,51159,Richmond,Richmond County,VA,Virginia,06,495925426.0,64458631.0,"POLYGON ((-76.93696 38.07707, -76.87706 38.12436, -76.76472 38.00198, -76.63423 37.96663, -76.55604 37.93152, -76.50783 37.83881, -76.56473 37.83729, -76.66818 37.77839, -76.78374 37.89548, -76.90903 37.98848, -76.93696 38.07707))"
178554,VA,Virginia,Richmond City,POINT (-77.4760085 37.531399),Richmond,51,760,01789073,0500000US51760,51760,Richmond,Richmond city,VA,Virginia,25,155205067.0,6849165.0,"POLYGON ((-77.59546 37.55575, -77.53239 37.59248, -77.53148 37.59179, -77.52309 37.58608, -77.51646 37.58167, -77.51620 37.58216, -77.51416 37.58157, -77.51325 37.58253, -77.49994 37.57540, -77.49353 37.58229, -77.49087 37.58012, -77.48237 37.57314, -77.47249 37.57869, -77.47385 37.58063, -77.47409 37.58181, -77.47722 37.58614, -77.47833 37.58756, -77.47921 37.58765, -77.47942 37.59081, -77.47913 37.59310, -77.47721 37.59790, -77.47815 37.59889, -77.47680 37.59854, -77.47090 37.59840, -77.46876 37.59862, -77.46606 37.59913, -77.46584 37.59676, -77.45448 37.59750, -77.45524 37.60178, -77.44758 37.60281, -77.44751 37.60222, -77.44361 37.60151, -77.43762 37.59869, -77.43762 37.59857, -77.38551 37.53517, -77.42074 37.44752, -77.59685 37.53465, -77.59546 37.55575))"
178555,VA,Virginia,Richmond City,POINT (-77.4760085 37.531399),Richmond,51,159,01480164,0500000US51159,51159,Richmond,Richmond County,VA,Virginia,06,495925426.0,64458631.0,"POLYGON ((-76.93696 38.07707, -76.87706 38.12436, -76.76472 38.00198, -76.63423 37.96663, -76.55604 37.93152, -76.50783 37.83881, -76.56473 37.83729, -76.66818 37.77839, -76.78374 37.89548, -76.90903 37.98848, -76.93696 38.07707))"
178634,VA,Virginia,Richmond City,POINT (-77.4760085 37.531399),Richmond,51,760,01789073,0500000US51760,51760,Richmond,Richmond city,VA,Virginia,25,155205067.0,6849165.0,"POLYGON ((-77.59546 37.55575, -77.53239 37.59248, -77.53148 37.59179, -77.52309 37.58608, -77.51646 37.58167, -77.51620 37.58216, -77.51416 37.58157, -77.51325 37.58253, -77.49994 37.57540, -77.49353 37.58229, -77.49087 37.58012, -77.48237 37.57314, -77.47249 37.57869, -77.47385 37.58063, -77.47409 37.58181, -77.47722 37.58614, -77.47833 37.58756, -77.47921 37.58765, -77.47942 37.59081, -77.47913 37.59310, -77.47721 37.59790, -77.47815 37.59889, -77.47680 37.59854, -77.47090 37.59840, -77.46876 37.59862, -77.46606 37.59913, -77.46584 37.59676, -77.45448 37.59750, -77.45524 37.60178, -77.44758 37.60281, -77.44751 37.60222, -77.44361 37.60151, -77.43762 37.59869, -77.43762 37.59857, -77.38551 37.53517, -77.42074 37.44752, -77.59685 37.53465, -77.59546 37.55575))"
178635,VA,Virginia,Richmond City,POINT (-77.4760085 37.531399),Richmond,51,159,01480164,0500000US51159,51159,Richmond,Richmond County,VA,Virginia,06,495925426.0,64458631.0,"POLYGON ((-76.93696 38.07707, -76.87706 38.12436, -76.76472 38.00198, -76.63423 37.96663, -76.55604 37.93152, -76.50783 37.83881, -76.56473 37.83729, -76.66818 37.77839, -76.78374 37.89548, -76.90903 37.98848, -76.93696 38.07707))"


In [35]:
merged_df2[merged_df2.LocationName=="Charles City"][['StateAbbr',	'StateDesc',	'LocationName',	'Geolocation',	'AdjustedLocationName']].drop_duplicates()

,StateAbbr,StateDesc,LocationName,Geolocation,AdjustedLocationName
402,VA,Virginia,Charles City,POINT (-77.05417 37.3610537),Charles


In [37]:
us_counties[us_counties.NAME=="Charles City"]

,STATEFP,COUNTYFP,COUNTYNS,AFFGEOID,GEOID,NAME,NAMELSAD,STUSPS,STATE_NAME,LSAD,ALAND,AWATER,geometry
775,51,036,01480109,0500000US51036,51036,Charles City,Charles City County,VA,Virginia,06,473711548,55274680,"POLYGON ((-77.24966 37.38200, -77.22427 37.38022, -77.17732 37.49060, -77.09581 37.44029, -76.90412 37.40382, -76.90447 37.37767, -76.87444 37.36543, -76.88316 37.22306, -76.92157 37.22534, -76.97244 37.24850, -76.99600 37.30098, -77.06927 37.26974, -77.12469 37.30292, -77.27067 37.32076, -77.24966 37.38200))"


In [38]:
def adjust_location_name(name):
    if isinstance(name, str):
        # Append 'County' to names that don't have 'City' and are not already suffixed with 'County'
        if " City" not in name and "County" not in name:
            return name + " County"
    return name

# Adjust location names in df_health
df_health['AdjustedLocationName'] = df_health['LocationName'].apply(adjust_location_name)

# Perform the merge
merged_df3 = pd.merge(df_health, us_counties, how='left', left_on=['AdjustedLocationName', 'StateAbbr'], right_on=['NAMELSAD', 'STUSPS'])


/tmp/ipykernel_54666/5252394.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_health['AdjustedLocationName'] = df_health['LocationName'].apply(adjust_location_name)


In [39]:
merged_df3[merged_df3.LocationName=="Charles City"][['StateAbbr',	'StateDesc',	'LocationName',	'Geolocation',	'AdjustedLocationName']].drop_duplicates()

,StateAbbr,StateDesc,LocationName,Geolocation,AdjustedLocationName
402,VA,Virginia,Charles City,POINT (-77.05417 37.3610537),Charles City


In [41]:
merged_df3[merged_df3.LocationName.str.contains("Richmond",na=False)][['StateAbbr',	'StateDesc',	'LocationName',	'Geolocation',	'AdjustedLocationName']].drop_duplicates()

,StateAbbr,StateDesc,LocationName,Geolocation,AdjustedLocationName
246,NY,New York,Richmond,POINT (-74.1398995 40.5612628),Richmond County
18686,GA,Georgia,Richmond,POINT (-82.0749982 33.3614863),Richmond County
107970,NC,North Carolina,Richmond,POINT (-79.7556946 35.004636),Richmond County
160335,VA,Virginia,Richmond,POINT (-76.7305609 37.9428936),Richmond County
160557,VA,Virginia,Richmond City,POINT (-77.4760085 37.531399),Richmond City


In [42]:
merged_df3[merged_df3.GEOID.isna()]

,StateAbbr,StateDesc,LocationName,Geolocation,AdjustedLocationName,STATEFP,COUNTYFP,COUNTYNS,AFFGEOID,GEOID,NAME,NAMELSAD,STUSPS,STATE_NAME,LSAD,ALAND,AWATER,geometry
0,US,United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
4,AK,Alaska,Aleutians East,POINT (-161.9974772 55.2450437),Aleutians East County,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
5,AK,Alaska,Anchorage,POINT (-149.2843294 61.1742503),Anchorage County,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
6,AK,Alaska,Dillingham,POINT (-158.2671226 59.5433255),Dillingham County,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
7,AK,Alaska,Fairbanks North Star,POINT (-146.5481586 64.6760436),Fairbanks North Star County,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
178080,VA,Virginia,Williamsburg,POINT (-76.7081925 37.269481),Williamsburg County,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
178082,VA,Virginia,Lexington,POINT (-79.4443196 37.7823316),Lexington County,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
178088,VA,Virginia,Hopewell,POINT (-77.2989443 37.2910098),Hopewell County,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
178092,VA,Virginia,Fredericksburg,POINT (-77.4866583 38.2992718),Fredericksburg County,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None


In [43]:
df_health[df_health.LocationName.str.contains("Richmond",na=False)][['StateAbbr',	'StateDesc',	'LocationName',	'Geolocation',	'AdjustedLocationName']].drop_duplicates()

,StateAbbr,StateDesc,LocationName,Geolocation,AdjustedLocationName
246,NY,New York,Richmond,POINT (-74.1398995 40.5612628),Richmond County
18686,GA,Georgia,Richmond,POINT (-82.0749982 33.3614863),Richmond County
107970,NC,North Carolina,Richmond,POINT (-79.7556946 35.004636),Richmond County
160335,VA,Virginia,Richmond,POINT (-76.7305609 37.9428936),Richmond County
160557,VA,Virginia,Richmond City,POINT (-77.4760085 37.531399),Richmond City


In [44]:
us_counties[us_counties.NAME.str.contains("Richmond")]

,STATEFP,COUNTYFP,COUNTYNS,AFFGEOID,GEOID,NAME,NAMELSAD,STUSPS,STATE_NAME,LSAD,ALAND,AWATER,geometry
355,37,153,01008581,0500000US37153,37153,Richmond,Richmond County,NC,North Carolina,06,1226849523,15575098,"POLYGON ((-80.07537 35.14308, -79.85127 35.18386, -79.61430 35.16368, -79.57593 35.06990, -79.45875 35.04364, -79.57342 34.99221, -79.56186 34.91035, -79.61405 34.81531, -79.69295 34.80496, -79.92468 34.80783, -79.84954 34.90001, -79.87720 35.03531, -79.92397 35.11543, -79.99294 35.08559, -80.07537 35.14308))"
901,36,085,00974141,0500000US36085,36085,Richmond,Richmond County,NY,New York,06,148982680,117441533,"POLYGON ((-74.24921 40.54506, -74.21684 40.55862, -74.20369 40.59269, -74.20225 40.63090, -74.20019 40.63184, -74.17061 40.64529, -74.16015 40.64608, -74.08681 40.65160, -74.07109 40.66706, -74.05575 40.65148, -74.05663 40.62729, -74.04139 40.60311, -74.03797 40.58957, -74.05732 40.59755, -74.11258 40.54760, -74.19992 40.51173, -74.26061 40.50244, -74.24921 40.54506))"
1191,13,245,00358024,0500000US13245,13245,Richmond,Richmond County,GA,Georgia,06,840036613,10995252,"POLYGON ((-82.35350 33.31232, -82.29418 33.35464, -82.28283 33.36270, -82.02824 33.54493, -82.01658 33.52909, -81.99094 33.49424, -81.92634 33.46294, -81.92012 33.41075, -81.93274 33.34354, -81.84614 33.30384, -81.84650 33.24725, -82.09713 33.23061, -82.17424 33.29677, -82.26758 33.26740, -82.33277 33.30868, -82.35350 33.31232))"
1580,51,760,01789073,0500000US51760,51760,Richmond,Richmond city,VA,Virginia,25,155205067,6849165,"POLYGON ((-77.59546 37.55575, -77.53239 37.59248, -77.53148 37.59179, -77.52309 37.58608, -77.51646 37.58167, -77.51620 37.58216, -77.51416 37.58157, -77.51325 37.58253, -77.49994 37.57540, -77.49353 37.58229, -77.49087 37.58012, -77.48237 37.57314, -77.47249 37.57869, -77.47385 37.58063, -77.47409 37.58181, -77.47722 37.58614, -77.47833 37.58756, -77.47921 37.58765, -77.47942 37.59081, -77.47913 37.59310, -77.47721 37.59790, -77.47815 37.59889, -77.47680 37.59854, -77.47090 37.59840, -77.46876 37.59862, -77.46606 37.59913, -77.46584 37.59676, -77.45448 37.59750, -77.45524 37.60178, -77.44758 37.60281, -77.44751 37.60222, -77.44361 37.60151, -77.43762 37.59869, -77.43762 37.59857, -77.38551 37.53517, -77.42074 37.44752, -77.59685 37.53465, -77.59546 37.55575))"
1979,51,159,01480164,0500000US51159,51159,Richmond,Richmond County,VA,Virginia,06,495925426,64458631,"POLYGON ((-76.93696 38.07707, -76.87706 38.12436, -76.76472 38.00198, -76.63423 37.96663, -76.55604 37.93152, -76.50783 37.83881, -76.56473 37.83729, -76.66818 37.77839, -76.78374 37.89548, -76.90903 37.98848, -76.93696 38.07707))"


## Merge by GEO

In [67]:
df_health = df_health[df_health.Geolocation.notna()]

In [68]:
# Convert the 'Geolocation' column to a GeoSeries
df_health['Geolocation'] = df_health['Geolocation'].apply(wkt.loads)
gdf_health = gpd.GeoDataFrame(df_health, geometry='Geolocation')


In [51]:
gdf_health.head()

,StateAbbr,StateDesc,LocationName,Geolocation,AdjustedLocationName
1,AL,Alabama,Talladega,POINT (-86.17593 33.36931),Talladega County
2,AL,Alabama,Tallapoosa,POINT (-85.79962 32.86331),Tallapoosa County
3,AL,Alabama,Wilcox,POINT (-87.30493 31.99008),Wilcox County
4,AK,Alaska,Aleutians East,POINT (-161.99748 55.24504),Aleutians East County
5,AK,Alaska,Anchorage,POINT (-149.28433 61.17425),Anchorage County


In [60]:
us_counties.crs

<Geographic 2D CRS: EPSG:4269>
Name: NAD83
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: North America - onshore and offshore: Canada - Alberta; British Columbia; Manitoba; New Brunswick; Newfoundland and Labrador; Northwest Territories; Nova Scotia; Nunavut; Ontario; Prince Edward Island; Quebec; Saskatchewan; Yukon. Puerto Rico. United States (USA) - Alabama; Alaska; Arizona; Arkansas; California; Colorado; Connecticut; Delaware; Florida; Georgia; Hawaii; Idaho; Illinois; Indiana; Iowa; Kansas; Kentucky; Louisiana; Maine; Maryland; Massachusetts; Michigan; Minnesota; Mississippi; Missouri; Montana; Nebraska; Nevada; New Hampshire; New Jersey; New Mexico; New York; North Carolina; North Dakota; Ohio; Oklahoma; Oregon; Pennsylvania; Rhode Island; South Carolina; South Dakota; Tennessee; Texas; Utah; Vermont; Virginia; Washington; West Virginia; Wisconsin; Wyoming. US Virgin Islands. British Virgin Islands

In [70]:
# Set the CRS of gdf_health to NAD83 (EPSG:4269)
gdf_health.set_crs(epsg=4269, inplace=True)

# Verify the CRS of both GeoDataFrames
print("CRS of gdf_health:", gdf_health.crs)
print("CRS of us_counties:", us_counties.crs)


CRS of gdf_health: EPSG:4269
CRS of us_counties: EPSG:4269


In [74]:
merged_gdf = gpd.sjoin(gdf_health, us_counties, how='left', predicate='intersects')

In [72]:
merged_gdf.head()

,StateAbbr,StateDesc,LocationName,Geolocation,index_right,STATEFP,COUNTYFP,COUNTYNS,AFFGEOID,GEOID,NAME,NAMELSAD,STUSPS,STATE_NAME,LSAD,ALAND,AWATER
1,AL,Alabama,Talladega,POINT (-86.17593 33.36931),2235.0,01,121,00161586,0500000US01121,01121,Talladega,Talladega County,AL,Alabama,06,1.908347e+09,6.087440e+07
2,AL,Alabama,Tallapoosa,POINT (-85.79962 32.86331),2108.0,01,123,00161587,0500000US01123,01123,Tallapoosa,Tallapoosa County,AL,Alabama,06,1.855798e+09,1.287910e+08
3,AL,Alabama,Wilcox,POINT (-87.30493 31.99008),2151.0,01,131,00161591,0500000US01131,01131,Wilcox,Wilcox County,AL,Alabama,06,2.299541e+09,5.059219e+07
4,AK,Alaska,Aleutians East,POINT (-161.99748 55.24504),2881.0,02,013,01419964,0500000US02013,02013,Aleutians East,Aleutians East Borough,AK,Alaska,04,1.809167e+10,2.079985e+10
5,AK,Alaska,Anchorage,POINT (-149.28433 61.17425),2389.0,02,020,01416061,0500000US02020,02020,Anchorage,Anchorage Municipality,AK,Alaska,12,4.420814e+09,6.210795e+08


In [75]:
merged_gdf[merged_gdf.LocationName.str.contains("Richmond")]

,StateAbbr,StateDesc,LocationName,Geolocation,index_right,STATEFP,COUNTYFP,COUNTYNS,AFFGEOID,GEOID,NAME,NAMELSAD,STUSPS,STATE_NAME,LSAD,ALAND,AWATER
246,NY,New York,Richmond,POINT (-74.13990 40.56126),901.0,36,085,00974141,0500000US36085,36085,Richmond,Richmond County,NY,New York,06,148982680.0,117441533.0
18686,GA,Georgia,Richmond,POINT (-82.07500 33.36149),1191.0,13,245,00358024,0500000US13245,13245,Richmond,Richmond County,GA,Georgia,06,840036613.0,10995252.0
19110,GA,Georgia,Richmond,POINT (-82.07500 33.36149),1191.0,13,245,00358024,0500000US13245,13245,Richmond,Richmond County,GA,Georgia,06,840036613.0,10995252.0
19185,GA,Georgia,Richmond,POINT (-82.07500 33.36149),1191.0,13,245,00358024,0500000US13245,13245,Richmond,Richmond County,GA,Georgia,06,840036613.0,10995252.0
19833,GA,Georgia,Richmond,POINT (-82.07500 33.36149),1191.0,13,245,00358024,0500000US13245,13245,Richmond,Richmond County,GA,Georgia,06,840036613.0,10995252.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
177572,VA,Virginia,Richmond City,POINT (-77.47601 37.53140),1580.0,51,760,01789073,0500000US51760,51760,Richmond,Richmond city,VA,Virginia,25,155205067.0,6849165.0
177722,VA,Virginia,Richmond,POINT (-76.73056 37.94289),1979.0,51,159,01480164,0500000US51159,51159,Richmond,Richmond County,VA,Virginia,06,495925426.0,64458631.0
177775,VA,Virginia,Richmond City,POINT (-77.47601 37.53140),1580.0,51,760,01789073,0500000US51760,51760,Richmond,Richmond city,VA,Virginia,25,155205067.0,6849165.0
177841,VA,Virginia,Richmond City,POINT (-77.47601 37.53140),1580.0,51,760,01789073,0500000US51760,51760,Richmond,Richmond city,VA,Virginia,25,155205067.0,6849165.0


In [76]:
len(df_health)

188396

In [77]:
len(merged_gdf)

188396

In [78]:
merged_gdf.head()

,StateAbbr,StateDesc,LocationName,Geolocation,index_right,STATEFP,COUNTYFP,COUNTYNS,AFFGEOID,GEOID,NAME,NAMELSAD,STUSPS,STATE_NAME,LSAD,ALAND,AWATER
1,AL,Alabama,Talladega,POINT (-86.17593 33.36931),2235.0,01,121,00161586,0500000US01121,01121,Talladega,Talladega County,AL,Alabama,06,1.908347e+09,6.087440e+07
2,AL,Alabama,Tallapoosa,POINT (-85.79962 32.86331),2108.0,01,123,00161587,0500000US01123,01123,Tallapoosa,Tallapoosa County,AL,Alabama,06,1.855798e+09,1.287910e+08
3,AL,Alabama,Wilcox,POINT (-87.30493 31.99008),2151.0,01,131,00161591,0500000US01131,01131,Wilcox,Wilcox County,AL,Alabama,06,2.299541e+09,5.059219e+07
4,AK,Alaska,Aleutians East,POINT (-161.99748 55.24504),2881.0,02,013,01419964,0500000US02013,02013,Aleutians East,Aleutians East Borough,AK,Alaska,04,1.809167e+10,2.079985e+10
5,AK,Alaska,Anchorage,POINT (-149.28433 61.17425),2389.0,02,020,01416061,0500000US02020,02020,Anchorage,Anchorage Municipality,AK,Alaska,12,4.420814e+09,6.210795e+08


## Cleaned version

In [79]:
df_health = pd.read_csv("../data/raw/CDC_PLACES_LOCAL/PLACES__Local_Data_for_Better_Health__County_Data_2022_release_20240102.csv")
# Load the US counties GeoJSON file using GeoPandas
us_counties_geojson_url = 'https://www2.census.gov/geo/tiger/GENZ2021/shp/cb_2021_us_county_20m.zip'
us_counties = gpd.read_file(us_counties_geojson_url)

In [80]:
df_health = df_health[df_health.Geolocation.notna()]

In [81]:
# Convert the 'Geolocation' column to a GeoSeries
df_health['Geolocation'] = df_health['Geolocation'].apply(wkt.loads)
gdf_health = gpd.GeoDataFrame(df_health, geometry='Geolocation')

In [82]:
# Set the CRS of gdf_health to NAD83 (EPSG:4269)
gdf_health.set_crs(epsg=4269, inplace=True)

# Verify the CRS of both GeoDataFrames
print("CRS of gdf_health:", gdf_health.crs)
print("CRS of us_counties:", us_counties.crs)

CRS of gdf_health: EPSG:4269
CRS of us_counties: EPSG:4269


In [96]:
merged_gdf = gpd.sjoin(gdf_health, us_counties, how='left', predicate='intersects')

In [84]:
merged_gdf.head(2)

,Year,StateAbbr,StateDesc,LocationName,DataSource,Category,Measure,Data_Value_Unit,Data_Value_Type,Data_Value,Data_Value_Footnote_Symbol,Data_Value_Footnote,Low_Confidence_Limit,High_Confidence_Limit,TotalPopulation,LocationID,CategoryID,MeasureId,DataValueTypeID,Short_Question_Text,Geolocation,index_right,STATEFP,COUNTYFP,COUNTYNS,AFFGEOID,GEOID,NAME,NAMELSAD,STUSPS,STATE_NAME,LSAD,ALAND,AWATER
1,2020,AL,Alabama,Talladega,BRFSS,Health Outcomes,Diagnosed diabetes among adults aged >=18 years,%,Crude prevalence,15.1,NaN,NaN,14.3,15.9,79985,1121,HLTHOUT,DIABETES,CrdPrv,Diabetes,POINT (-86.17593 33.36931),2235.0,01,121,00161586,0500000US01121,01121,Talladega,Talladega County,AL,Alabama,06,1.908347e+09,60874403.0
2,2020,AL,Alabama,Tallapoosa,BRFSS,Health Outcomes,Diagnosed diabetes among adults aged >=18 years,%,Crude prevalence,16.0,NaN,NaN,15.0,17.0,40133,1123,HLTHOUT,DIABETES,CrdPrv,Diabetes,POINT (-85.79962 32.86331),2108.0,01,123,00161587,0500000US01123,01123,Tallapoosa,Tallapoosa County,AL,Alabama,06,1.855798e+09,128790974.0


In [86]:
merged_gdf = merged_gdf[merged_gdf.Year==2020]

In [99]:
# let's use only Age-adjusted prevalence
merged_gdf = merged_gdf[merged_gdf.Data_Value_Type!='Crude prevalence']

In [100]:
merged_gdf = merged_gdf[['StateAbbr','StateDesc','LocationName','Measure','Data_Value','Data_Value_Unit','GEOID']]
merged_gdf.to_pickle("../data/interim/health_food_2020_GEOID.pickle")

In [ ]:
# df_health_food_2020 = df_health_food_2020[['StateAbbr','StateDesc','LocationName','Measure','Data_Value','Data_Value_Unit','FIPS']]
# df.to_pickle("../data/interim/health_food_2020.pickle")

In [101]:
merged_gdf[(merged_gdf.LocationName=='Summit') & (merged_gdf.StateAbbr=='UT') & (merged_gdf.Measure=='Binge drinking among adults aged >=18 years')]

,StateAbbr,StateDesc,LocationName,Measure,Data_Value,Data_Value_Unit,GEOID
177335,UT,Utah,Summit,Binge drinking among adults aged >=18 years,13.6,%,49043


In [103]:
merged_gdf[(merged_gdf.LocationName=='Utah') & (merged_gdf.StateAbbr=='UT') & (merged_gdf.Measure=='Binge drinking among adults aged >=18 years')]

,StateAbbr,StateDesc,LocationName,Measure,Data_Value,Data_Value_Unit,GEOID
171717,UT,Utah,Utah,Binge drinking among adults aged >=18 years,8.2,%,49049


In [108]:
test_mg = merged_gdf[(merged_gdf.Measure=='Stroke among adults aged >=18 years')]

In [109]:
test_mg

,StateAbbr,StateDesc,LocationName,Measure,Data_Value,Data_Value_Unit,GEOID
8,AK,Alaska,Haines,Stroke among adults aged >=18 years,2.9,%,02100
41,CO,Colorado,Bent,Stroke among adults aged >=18 years,3.5,%,08011
42,CO,Colorado,Broomfield,Stroke among adults aged >=18 years,1.9,%,08014
81,IL,Illinois,Champaign,Stroke among adults aged >=18 years,2.8,%,17019
92,IL,Illinois,Montgomery,Stroke among adults aged >=18 years,3.0,%,17135
...,...,...,...,...,...,...,...
188123,WI,Wisconsin,Grant,Stroke among adults aged >=18 years,2.8,%,55043
188156,WI,Wisconsin,Polk,Stroke among adults aged >=18 years,2.5,%,55095
188200,WV,West Virginia,Roane,Stroke among adults aged >=18 years,3.7,%,54087
188240,WI,Wisconsin,Rusk,Stroke among adults aged >=18 years,3.0,%,55107


In [110]:
# Count duplicates for each 'Measure' based on 'StateAbbr', 'LocationName', and 'Measure'
duplicate_counts = merged_gdf[merged_gdf.duplicated(subset=['StateAbbr', 'LocationName', 'Measure'], keep=False)].groupby('Measure').size()

print(duplicate_counts)


Series([], dtype: int64)
